In [7]:
import geopandas as gpd
import os

# Read the shapefile
shape_file = gpd.read_file(r"Data/Aus_State_Shapefile/STE_2021_AUST_GDA2020.shp")

print(shape_file.head())


  STE_CODE21         STE_NAME21 CHG_FLAG21  CHG_LBL21 AUS_CODE21 AUS_NAME21  \
0          1    New South Wales          0  No change        AUS  Australia   
1          2           Victoria          0  No change        AUS  Australia   
2          3         Queensland          0  No change        AUS  Australia   
3          4    South Australia          0  No change        AUS  Australia   
4          5  Western Australia          0  No change        AUS  Australia   

     AREASQKM21                                       LOCI_URI21  \
0  8.007977e+05  http://linked.data.gov.au/dataset/asgsed3/STE/1   
1  2.274962e+05  http://linked.data.gov.au/dataset/asgsed3/STE/2   
2  1.730171e+06  http://linked.data.gov.au/dataset/asgsed3/STE/3   
3  9.842314e+05  http://linked.data.gov.au/dataset/asgsed3/STE/4   
4  2.526632e+06  http://linked.data.gov.au/dataset/asgsed3/STE/5   

                                            geometry  
0  MULTIPOLYGON (((159.0623 -31.50886, 159.06218 ...  
1  MUL

In [8]:
# Read the shapefile
shape_file_tas = gpd.read_file(r"Data\Tas_Shapefile\tasmania.shp")

print(shape_file_tas.head())

  STE_CODE21 STE_NAME21 CHG_FLAG21  CHG_LBL21 AUS_CODE21 AUS_NAME21  \
0          6   Tasmania          0  No change        AUS  Australia   

   AREASQKM21                                       LOCI_URI21  \
0  68017.5393  http://linked.data.gov.au/dataset/asgsed3/STE/6   

                                            geometry  
0  MULTIPOLYGON (((144.60439 -41.01001, 144.60443...  


In [4]:
import geopandas as gpd
import os

# Read the shapefile
shape_file = gpd.read_file(r"Data/Aus_State_Shapefile/STE_2021_AUST_GDA2020.shp")

# Filter only Tasmania
tas_shape = shape_file[shape_file['STE_NAME21'] == 'Tasmania']

# Create output folder (if not exists)
os.makedirs("Data/Tas_Shapefile", exist_ok=True)

# Save Tasmania shapefile correctly with a filename
tas_shape.to_file(r"Data/Tas_Shapefile/tasmania.shp")


In [9]:
import os
import xarray as xr
import geopandas as gpd
import rioxarray  # important: enables .rio

# ========= 1. PATHS =========
input_folder = r"Data\Farm Data\2_Cleaned_Aus_Farm_Data"            # your input .nc folder
output_folder = r"Data\Farm Data\3_Cleaned_Tas_Farm_Data"   # output folder
shapefile_path = r"Data\Tasmania_Boundary\tasmania_boundary.shp"  # <-- update

os.makedirs(output_folder, exist_ok=True)

# ========= 2. READ SHAPEFILE =========
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs("EPSG:4326")
shapes = gdf.geometry.values

# ========= 3. LOOP THROUGH FILES =========
for fname in os.listdir(input_folder):
    if not fname.endswith(".nc"):
        continue

    in_path = os.path.join(input_folder, fname)
    base = fname[:-3]  # strip ".nc"
    out_path = os.path.join(output_folder, f"{base}_cropped.nc")

    print(f"Processing: {fname}")

    # Open dataset
    ds = xr.open_dataset(in_path)

    # ---- Tell rioxarray which dims are spatial ----
    # ABARES grid uses dims 'lon' and 'lat'
    # If your file uses something slightly different, print(ds.dims) to check.
    ds = ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=False)

    # Ensure CRS is set
    if ds.rio.crs is None:
        ds = ds.rio.write_crs("EPSG:4326")

    # ---- Clip ----
    try:
        ds_clipped = ds.rio.clip(
            shapes,
            ds.rio.crs,
            drop=True,
            from_disk=True
        )
    except Exception as e:
        print(f"  ⚠ Error clipping {fname}: {e}")
        ds.close()
        continue

    # Save clipped dataset
    ds_clipped.to_netcdf(out_path)

    ds.close()
    ds_clipped.close()

    print(f"  ✔ Saved: {out_path}")

print("All files clipped and saved with _cropped suffix.")


Processing: Aus_farm_data_1991.nc
  ✔ Saved: Data\Farm Data\3_Cleaned_Tas_Farm_Data\Aus_farm_data_1991_cropped.nc
Processing: Aus_farm_data_1992.nc
  ✔ Saved: Data\Farm Data\3_Cleaned_Tas_Farm_Data\Aus_farm_data_1992_cropped.nc
Processing: Aus_farm_data_1993.nc
  ✔ Saved: Data\Farm Data\3_Cleaned_Tas_Farm_Data\Aus_farm_data_1993_cropped.nc
Processing: Aus_farm_data_1994.nc
  ✔ Saved: Data\Farm Data\3_Cleaned_Tas_Farm_Data\Aus_farm_data_1994_cropped.nc
Processing: Aus_farm_data_1995.nc
  ✔ Saved: Data\Farm Data\3_Cleaned_Tas_Farm_Data\Aus_farm_data_1995_cropped.nc
Processing: Aus_farm_data_1996.nc
  ✔ Saved: Data\Farm Data\3_Cleaned_Tas_Farm_Data\Aus_farm_data_1996_cropped.nc
Processing: Aus_farm_data_1997.nc
  ✔ Saved: Data\Farm Data\3_Cleaned_Tas_Farm_Data\Aus_farm_data_1997_cropped.nc
Processing: Aus_farm_data_1998.nc
  ✔ Saved: Data\Farm Data\3_Cleaned_Tas_Farm_Data\Aus_farm_data_1998_cropped.nc
Processing: Aus_farm_data_1999.nc
  ✔ Saved: Data\Farm Data\3_Cleaned_Tas_Farm_Data\Aus_